# File Operations: Basin, Met, Control, and Gage

This notebook demonstrates direct file operations for HMS component files:

| Class | File Type | Purpose |
|-------|-----------|--------|
| `HmsBasin` | `.basin` | Read/modify subbasins, reaches, junctions |
| `HmsMet` | `.met` | Read/modify gage assignments and precipitation methods |
| `HmsControl` | `.control` | Read/modify simulation time windows and intervals |
| `HmsGage` | `.gage` | Read gage definitions and DSS references |

These classes operate directly on HMS files without requiring project initialization, making them useful for batch processing and standalone file operations.

**Estimated Time**: 15-20 minutes

In [1]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

## IMPORTANT: File Modification Warning

**This notebook modifies HMS project files directly!**

When you use `set_*` methods (like `HmsBasin.set_loss_parameters()`), changes are:
- **Written to disk immediately** - no undo button
- **Persistent** - changes remain after notebook restart
- **Cumulative** - multiple runs add up changes

### Best Practices

1. **Always work on extracted copies** (this notebook does this with `HmsExamples.extract_project()`)
2. **Use clone workflows** for QAQC-grade work (see notebook 05)
3. **Record original values** before modifying (shown in examples below)
4. **Re-extract project** to reset to original state

### To Reset This Project to Original State

```python
# Delete and re-extract to get fresh copy
import shutil
shutil.rmtree('hms_example_projects/castro_file_ops', ignore_errors=True)
HmsExamples.extract_project('castro', output_path='hms_example_projects/castro_file_ops')
```

In [2]:
from pathlib import Path
from hms_commander import (
    HmsExamples,
    init_hms_project,
    HmsBasin,
    HmsMet,
    HmsControl,
    HmsGage
)

print("hms-commander loaded")

hms-commander loaded


## 1. Extract Example Project

We use the `castro` project which has multiple subbasins, reaches, and gages.

In [3]:
# Extract the castro example project
project_path = HmsExamples.extract_project(
    "castro",
    output_path=Path.cwd() / 'hms_example_projects' / 'castro_file_ops'
)

print(f"Project extracted to: {project_path}")

# Initialize project for reference (optional - file ops work without this)
hms = init_hms_project(project_path)
print(f"Project: {hms.project_name}, Version: {hms.hms_version}")

2026-01-08 13:16:56 - hms_commander.HmsExamples - INFO - Found HMS 4.10 at C:\Program Files\HEC\HEC-HMS\4.10


2026-01-08 13:16:56 - hms_commander.HmsExamples - INFO - Found HMS 4.11 at C:\Program Files\HEC\HEC-HMS\4.11


2026-01-08 13:16:56 - hms_commander.HmsExamples - INFO - Found HMS 4.12 at C:\Program Files\HEC\HEC-HMS\4.12


2026-01-08 13:16:56 - hms_commander.HmsExamples - INFO - Found HMS 4.13 at C:\Program Files\HEC\HEC-HMS\4.13


2026-01-08 13:16:56 - hms_commander.HmsExamples - INFO - Found HMS 4.4.1 at C:\Program Files\HEC\HEC-HMS\4.4.1


...

2026-01-08 13:16:56 - hms_commander.HmsPrj - INFO -   Control specs: 1


2026-01-08 13:16:56 - hms_commander.HmsPrj - INFO -   Simulation runs: 2


2026-01-08 13:16:56 - hms_commander.HmsPrj - INFO -   Gages: 2


2026-01-08 13:16:56 - hms_commander.HmsPrj - INFO -   Paired data tables: 1


Project extracted to: C:\GH\hms-commander\examples\hms_example_projects\castro_file_ops\castro
Project: castro, Version: 4.13


## HMS File Naming Conventions

HMS projects follow a consistent naming convention:

| File Extension | Contents | Example |
|---------------|----------|--------|
| `.hms` | Project definition | `castro.hms` |
| `.basin` | Basin model (subbasins, reaches, junctions) | `castro.basin` |
| `.met` | Meteorologic model (precipitation config) | `castro.met` |
| `.control` | Control specifications (time window) | `castro.control` |
| `.gage` | Gage definitions (time series references) | `castro.gage` |
| `.run` | Run configurations | `castro.run` |
| `.dss` | Results/data storage | `castro.dss` |

**Note**: A project can have multiple basin/met/control files with different names for different scenarios.

## 2. Basin File Operations (HmsBasin)

`HmsBasin` provides methods to read and modify `.basin` files:
- `get_subbasins()` - List all subbasins with areas and methods
- `get_junctions()` - List all junctions
- `get_reaches()` - List all reaches with routing methods
- `get_loss_parameters()` / `set_loss_parameters()` - Loss method parameters
- `get_transform_parameters()` - Unit hydrograph parameters

In [4]:
# Get the first basin file
basin_file = hms.basin_df['full_path'].iloc[0]
print(f"Basin file: {Path(basin_file).name}")
print("=" * 60)

Basin file: Castro_1.basin


### Read Subbasins

In [5]:
# Get all subbasins from the basin file
subbasins = HmsBasin.get_subbasins(basin_file)

print(f"Found {len(subbasins)} subbasins:")
print("=" * 60)

# Display key columns
display_cols = ['name', 'area', 'downstream', 'loss_method', 'transform_method']
available_cols = [c for c in display_cols if c in subbasins.columns]
subbasins[available_cols]

2026-01-08 13:16:56 - hms_commander.HmsBasin - INFO - Reading subbasins from: C:\GH\hms-commander\examples\hms_example_projects\castro_file_ops\castro\Castro_1.basin


2026-01-08 13:16:56 - hms_commander.HmsBasin - INFO - Found 4 subbasins


Found 4 subbasins:


,name,area,downstream,loss_method,transform_method
0,Subbasin-3,2.17,Reach-2,None,Snyder
1,Subbasin-4,0.96,West Branch,None,Snyder
2,Subbasin-1,0.86,Reach-1,None,Snyder
3,Subbasin-2,1.52,East Branch,None,Snyder


### Read Junctions and Reaches

In [6]:
# Get junctions
junctions = HmsBasin.get_junctions(basin_file)
print(f"Found {len(junctions)} junctions:")
print(junctions['name'].tolist())

# Get reaches
reaches = HmsBasin.get_reaches(basin_file)
print(f"\nFound {len(reaches)} reaches:")
display_cols = ['name', 'downstream', 'routing_method']
available_cols = [c for c in display_cols if c in reaches.columns]
display(reaches[available_cols])

Found 3 junctions:
['West Branch', 'East Branch', 'Outlet']

Found 2 reaches:


,name,downstream
0,Reach-2,West Branch
1,Reach-1,East Branch


### Read and Modify Loss Parameters

**Common Loss Parameters by Method**:

| Method | Key Parameters |
|--------|---------------|
| SCS Curve Number | `curve_number`, `percent_impervious`, `initial_abstraction` |
| Green and Ampt | `saturated_conductivity`, `suction`, `initial_deficit` |
| Initial + Constant | `initial_loss`, `constant_rate`, `percent_impervious` |

In [7]:
# Get loss parameters for first subbasin
subbasin_name = subbasins['name'].iloc[0]
print(f"Loss parameters for '{subbasin_name}':")
print("=" * 60)

loss_params = HmsBasin.get_loss_parameters(basin_file, subbasin_name)
for key, value in loss_params.items():
    print(f"  {key}: {value}")

Loss parameters for 'Subbasin-3':
  method: None
  percent_impervious: 10.0


In [8]:
# Modify loss parameters (example: update percent impervious)
# REMEMBER: This modifies the actual file on disk!

if 'percent_impervious' in loss_params:
    # Record original value for potential revert
    original_value = loss_params['percent_impervious']
    new_value = original_value + 5.0  # Increase by 5%
    
    print(f"Original value: {original_value}")
    print(f"Modifying percent_impervious: {original_value} -> {new_value}")

    HmsBasin.set_loss_parameters(
        basin_file,
        subbasin_name,
        percent_impervious=new_value
    )

    # Verify change
    updated_params = HmsBasin.get_loss_parameters(basin_file, subbasin_name)
    print(f"Verified new value: {updated_params['percent_impervious']}")
    print("\n[!] Change persisted to disk. See warning above for reset instructions.")
else:
    print("Subbasin does not use percent_impervious parameter")

2026-01-08 13:16:56 - hms_commander.HmsBasin - INFO - Updated loss parameters for subbasin 'Subbasin-3'


Original value: 10.0
Modifying percent_impervious: 10.0 -> 15.0
Verified new value: 15.0

[!] Change persisted to disk. See warning above for reset instructions.


### Read Transform Parameters

In [9]:
# Get transform (unit hydrograph) parameters
print(f"Transform parameters for '{subbasin_name}':")
print("=" * 60)

transform_params = HmsBasin.get_transform_parameters(basin_file, subbasin_name)
for key, value in transform_params.items():
    print(f"  {key}: {value}")

Transform parameters for 'Subbasin-3':
  method: Snyder
  snyder_tp: 0.2
  snyder_cp: 0.16


## 3. Met File Operations (HmsMet)

`HmsMet` provides methods to read and modify `.met` files:
- `get_mets()` - List all met models in a file
- `get_precipitation_method()` - Get precip method type
- `get_gage_assignments()` - Get subbasin-to-gage mappings
- `set_gage_assignment()` - Update a subbasin's gage assignment
- `get_dss_references()` - Get DSS file references

In [10]:
# Get the first met file
met_file = hms.met_df['full_path'].iloc[0]
print(f"Met file: {Path(met_file).name}")
print("=" * 60)

Met file: GageWts.met


### Read Precipitation Method

In [11]:
# Get precipitation method
met_name = hms.met_df['name'].iloc[0]
precip_method = HmsMet.get_precipitation_method(met_file, met_name)

print(f"Met model: {met_name}")
print(f"Precipitation method: {precip_method}")

Met model: GageWts
Precipitation method: None


### Read Gage Assignments

In [12]:
# Get gage assignments (which gage is assigned to each subbasin)
# Returns: DataFrame with columns ['subbasin', 'precip_gage', 'weight']
assignments = HmsMet.get_gage_assignments(met_file, met_name)

print(f"Gage assignments for '{met_name}':")
print("=" * 60)
display(assignments)

2026-01-08 13:16:56 - hms_commander.HmsMet - INFO - Reading gage assignments from: C:\GH\hms-commander\examples\hms_example_projects\castro_file_ops\castro\GageWts.met


2026-01-08 13:16:56 - hms_commander.HmsMet - INFO - Found 4 gage assignments


Gage assignments for 'GageWts':


,subbasin,precip_gage,weight
0,Subbasin-1,None,1.0
1,Subbasin-2,None,1.0
2,Subbasin-3,None,1.0
3,Subbasin-4,None,1.0


### Read DSS References

In [13]:
# Get DSS file references in the met model
dss_refs = HmsMet.get_dss_references(met_file, met_name)

print(f"DSS references in '{met_name}':")
print("=" * 60)
if dss_refs:
    for ref in dss_refs:
        print(f"  {ref}")
else:
    print("  No DSS references found (may use direct gage data)")

DSS references in 'GageWts':
  No DSS references found (may use direct gage data)


## 4. Control File Operations (HmsControl)

`HmsControl` provides methods to read and modify `.control` files:
- `get_controls()` - List all control specs in a file
- `get_time_window()` - Get start/end dates and times
- `set_time_window()` - Update simulation time window
- `get_time_interval()` - Get computation interval
- `set_time_interval()` - Update computation interval

In [14]:
# Get the first control file
control_file = hms.control_df['full_path'].iloc[0]
print(f"Control file: {Path(control_file).name}")
print("=" * 60)

Control file: Jan73.control


### Read Time Window

In [15]:
# Get time window for the control spec
control_name = hms.control_df['name'].iloc[0]
time_window = HmsControl.get_time_window(control_file)

print(f"Time window for '{control_name}':")
print("=" * 60)
for key, value in time_window.items():
    print(f"  {key}: {value}")

2026-01-08 13:16:56 - hms_commander.HmsControl - INFO - Reading time window from: C:\GH\hms-commander\examples\hms_example_projects\castro_file_ops\castro\Jan73.control


ValueError: Error parsing date/time: time data '16 January 1973 03:00' does not match format '%d%b%Y %H:%M'

### Read Time Interval

In [ ]:
# Get computation time interval
interval = HmsControl.get_time_interval(control_file)

print(f"Time interval for '{control_name}':")
print("=" * 60)
print(f"  Interval: {interval}")

### Get Full Control Info

In [ ]:
# Get complete control specification info
control_info = HmsControl.get_control_info(control_file)

print(f"Complete info for '{control_name}':")
print("=" * 60)
for key, value in control_info.items():
    print(f"  {key}: {value}")

## 5. Gage File Operations (HmsGage)

`HmsGage` provides methods to read `.gage` files:
- `get_gages()` - List all gages in a project
- `get_gage_info()` - Get detailed info for a specific gage
- `get_dss_pathname()` - Get DSS pathname for a gage
- `list_precip_gages()` / `list_discharge_gages()` - Filter by gage type

In [ ]:
# Get gage file path from project
gage_file = project_path / (project_path.name + ".gage")
print(f"Gage file: {gage_file.name}")
print(f"File exists: {gage_file.exists()}")
print("=" * 60)

### List All Gages

In [ ]:
# Get all gages from the gage file
# Note: If the gage file doesn't exist, we fall back to gage_df from project init.
# This silent fallback is intentional - HMS projects may store gage info in different files.
if gage_file.exists():
    gages = HmsGage.get_gages(str(gage_file))

    print(f"Found {len(gages)} gages:")
    print("=" * 60)

    display_cols = ['name', 'gage_type', 'dss_file']
    available_cols = [c for c in display_cols if c in gages.columns]
    display(gages[available_cols])
else:
    # Fallback: use project gage_df (parsed during project initialization)
    print(f"Gage file not found at: {gage_file}")
    print("Using project gage DataFrame (from hms.gage_df):")
    display(hms.gage_df[['name', 'gage_type', 'dss_file']])

### Get DSS Pathnames for Gages

In [ ]:
# Get DSS pathnames for all gages
print("DSS Pathnames:")
print("=" * 60)

for idx, row in hms.gage_df.iterrows():
    gage_name = row['name']
    gage_type = row['gage_type']
    
    # Try to get DSS pathname
    if 'dss_pathname' in row:
        pathname = row['dss_pathname']
    else:
        pathname = 'N/A'
    
    print(f"  {gage_name} ({gage_type}):")
    print(f"    {pathname}")

### Filter Gages by Type

In [ ]:
# Use project accessor methods to filter gages
precip_gages = hms.list_gage_names(gage_type='Precipitation')
flow_gages = hms.list_gage_names(gage_type='Flow')

print(f"Precipitation gages ({len(precip_gages)}): {precip_gages}")
print(f"Flow gages ({len(flow_gages)}): {flow_gages}")

## 6. Direct File Access (Without Project Initialization)

All file operations can be performed directly on files without initializing a project. This is useful for batch processing.

In [ ]:
# Example: Read basin elements directly from file path
# No init_hms_project() needed!

# Get basin filename dynamically from project DataFrame (recommended)
# Note: The column is 'file_name' (with underscore), not 'filename'
basin_filename = hms.basin_df['file_name'].iloc[0]
basin_path = project_path / basin_filename

if basin_path.exists():
    # Direct file read
    direct_subbasins = HmsBasin.get_subbasins(str(basin_path))
    direct_junctions = HmsBasin.get_junctions(str(basin_path))
    direct_reaches = HmsBasin.get_reaches(str(basin_path))

    print(f"Direct file access (no project init):")
    print(f"  Basin file: {basin_filename}")
    print(f"  Subbasins: {len(direct_subbasins)}")
    print(f"  Junctions: {len(direct_junctions)}")
    print(f"  Reaches: {len(direct_reaches)}")
else:
    print(f"Basin file not found at: {basin_path}")

## 7. Reverting Changes

Since file operations modify files directly, here are your options to revert:

### Option 1: Re-extract the Project (Recommended)

```python
import shutil

# Delete modified project
shutil.rmtree('hms_example_projects/castro_file_ops', ignore_errors=True)

# Extract fresh copy
project_path = HmsExamples.extract_project(
    'castro',
    output_path='hms_example_projects/castro_file_ops'
)
```

### Option 2: Restore Specific Value

```python
# If you recorded original value:
HmsBasin.set_loss_parameters(
    basin_file,
    subbasin_name,
    percent_impervious=original_value  # Restore original
)
```

### Option 3: Use Clone Workflow (Proactive)

For production work, use `HmsBasin.clone_basin()` to create a copy first:

```python
# Create cloned basin before modifications
cloned_basin = HmsBasin.clone_basin(basin_file, 'Castro_Modified')

# Modify the clone, original is preserved
HmsBasin.set_loss_parameters(cloned_basin, subbasin_name, percent_impervious=25.0)
```

See **05_clone_workflow.ipynb** for complete clone workflow.

## Summary

This notebook demonstrated file operations for the four main HMS file types:

| Class | Key Methods | Use Case |
|-------|-------------|----------|
| `HmsBasin` | `get_subbasins`, `get_loss_parameters`, `set_loss_parameters` | Modify basin parameters |
| `HmsMet` | `get_precipitation_method`, `get_gage_assignments` | Read met model config |
| `HmsControl` | `get_time_window`, `set_time_window` | Adjust simulation periods |
| `HmsGage` | `get_gages`, `get_dss_pathname` | Read gage definitions |

**Key Points**:
- All classes use **static methods** - no instantiation needed
- Operations work **directly on files** - project init is optional
- **Modifications persist** to disk immediately
- Useful for **batch processing** across multiple projects
- **Use clones** for QAQC workflows to preserve originals

## Next Steps

- **04_run_management.ipynb**: Configure simulation runs with validation
- **05_clone_workflow.ipynb**: Non-destructive cloning for QAQC
- **06_results_dss.ipynb**: Extract results from DSS files